In [ ]:
#Parameters

## rte
multithreaded=True
tycheplotPath="../../TychePlot/"

## files
series_indicator=""
series_name=""
jvl_subdir=""
spectra_subdir=""


# names
labelList=[  
                "A",
                "B",
                "C",
                "D"
          ]
# spectra
spectra=[('XLVII',2)] # Give one for all devices or for each device one spectrum. syntax: (filename, column)

# valid EQE threshold

eqe_thresh=0.5 #cd/m2

In [ ]:
import sys
import os
sys.path.append(tycheplotPath)
%matplotlib inline
import matplotlib.pyplot as plt
from Plot_processing import calc, export_data, calc_plot_list
from PDF import PDF
from OLEDPlot import OLEDPlot
from Filereader import fileToNpArray
import dill
import copy

#Constants
Voltage=1
Current=2
Current_density=3
Luminance=4
Radiance=5
Current_Efficacy=6
Luminous_Efficacy=7
EQE=8
POWER=9

valid_pixel_file="valid_pixel.bin"
fill="_"
jvl_file_format=OLEDPlot.jvl_file_format_default
spectra_file_format=OLEDPlot.spectral_data_format_default

#Dynamic Parameters
name=series_indicator+fill+series_name

#stub:
desiredPlots=[
    {
        "xCol":Voltage,
        "yCol":Current_density,   
}  
]

try:
    spectra_new=[]
    for spectrum in spectra:
        spectrum=list(spectrum)
        spectrum[0]=spectra_subdir+spectrum[0]
        spectra_new.append(tuple(spectrum))
    spectra=spectra_new
except TypeError:
    spectra_list=list(spectra)
    spectra_list[0]=spectra_subdir+spectra_list[0]
    spectra=tuple(spectra_list)

In [ ]:
inputParameters={
                    "spectraFile":spectra,
                    "labels":labelList,
                    "pixelsize_mm2":4, #mm2
                    "averageMedian":True,
                    "noSweepBackMeasured":True,
                    "no_plot":True,
                    "spectralDataFormat":{"separator":"\t", "skiplines":53, "fileEnding":".txt", "codec":"iso-8859-1"}
                    "fileFormat":{"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True},
                }

In [ ]:
#getting valid_pixels (file or user_input)
try:
    with open(jvl_subdir+valid_pixel_file, "rb") as f:
        valid_info=dill.load(f)
except FileNotFoundError:
    valid_info=OLEDPlot.get_valid_pixel_by_user(series_indicator, subdir=jvl_subdir, update_by_existing=True, jvl_file_format={"separator":"\t", "skiplines":1, "fileEnding":".uil", "commaToPoint":True})
    with open(jvl_subdir+valid_pixel_file, "wb") as f:
        dill.dump(valid_info, f)
valid_pixel=valid_info[1]
valid_device=valid_info[0]

In [ ]:
#Importing files
OLED_fileList=OLEDPlot.generateFileList(series_indicator, subdir=jvl_subdir, truthTable=valid_pixel, update_by_existing=True)
OLED_fileList=sorted(OLED_fileList, key=lambda OLED_file: OLED_file[0][-3:])
OLED_fileList

In [ ]:
init_plot_args=(name, OLED_fileList, inputParameters, OLEDPlot, None)

In [ ]:
plot, files=calc(desiredPlots, init_plot_args, multithreaded=multithreaded, exportonly=True)[0]
files

In [ ]:
max_eqes=[]
for data, label in zip(plot.expectData,labelList):
    try:
        valid_eqe_start=data.getFirstIndexWhereGreaterOrEq(4,eqe_thresh, check_seq=5)
        valid_eqes=data.getData()[:,7][valid_eqe_start:]
        max_eqe=max(valid_eqes)
        max_eqes.append(max_eqe)
        print(f"Device {label}: Max EQE: {max_eqe:.3f} %")
    except IndexError:
        max_eqes.append(0)
        print(f"Device {label}: No EQE obtained, too low luminace!")